In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

In [2]:
class OrderParameter:

    # name should be unique to the Order Parameter being defined
    # In other words for every possible pair of OP's x and y, (x.name != y.name) must be true
    def __init__(self, name, traj):
        self.name = name
        self.traj = traj
        
    def __eq__(self, other):
        return self.name == other.name
    
    def __hash__(self):
        return hash(self.name)
        
    def __str__(self):
        return str(self.name)

In [3]:
def distortion(centers, ops, mut):
    dis = 0.0
    for i in ops:
        min_val = np.inf
        for j in centers:
            tmp = mut.iqr(i, j)
            if tmp < min_val:
                min_val = tmp
        dis = dis + (min_val * min_val)
    return dis ** (0.5)

In [4]:
class DissimilarityMatrix:

    def __init__(self, max_OPs, mut):
        self.max_OPs = max_OPs
        self.matrix = [[] for i in range(max_OPs)]
        self.mut = mut
        self.OPs = []

    def add_OP(self, OP):
        if len(self.OPs) == self.max_OPs:
            mut_info = []
            existing = []
            for i in range(len(self.OPs)):
                mut_info.append(self.mut.iqr(self.OPs[i], OP))
                product = 1
                for j in range(len(self.OPs)):
                    if not i == j:
                        product = product * self.matrix[i][j]
                existing.append(product)
            update = False
            difference = None
            for i in range(len(self.OPs)):
                candidate_info = 1
                for j in range(len(self.OPs)):
                    if not i == j:
                        candidate_info = candidate_info * mut_info[j]
                if candidate_info > existing[i]:
                    update = True
                    if difference == None:
                        difference = candidate_info - existing[i]
                        old_OP = i
                    else:
                        if (candidate_info - existing[i]) > difference:
                            difference = candidate_info - existing[i]
                            old_OP = i
            if update == True:
                mut_info[old_OP] = self.mut.iqr(OP, OP)
                self.matrix[old_OP] = mut_info
                self.OPs[old_OP] = OP
                for i in range(len(self.OPs)):
                    self.matrix[i][old_OP] = mut_info[i]
        else:
            for i in range(len(self.OPs)):
                mut_info = self.mut.iqr(OP, self.OPs[i])
                self.matrix[i].append(mut_info)
                self.matrix[len(self.OPs)].append(mut_info)
            self.matrix[len(self.OPs)].append(self.mut.iqr(OP, OP))
            self.OPs.append(OP)
        
    def reduce(self):
        min_val = 10
        index = -1
        for i in range(len(self.matrix)):
            product = 1
            for j in range(len(self.matrix[i])):
                if not i == j:
                    product = product * self.matrix[i][j]
            if product < min_val:
                index = i
                min_val = product
        self.matrix.pop(index)
        for i in range(len(self.matrix)):
            self.matrix[i].pop(index)
        self.OPs.pop(index)
        
    def min_product(self):
        min_val = 10
        for i in range(len(self.matrix)):
            product = 1
            for j in range(len(self.matrix[i])):
                if not i == j:
                    product = product * self.matrix[i][j]
            if product < min_val:
                min_val = product
        return min_val
    
    def get_OPs(self):
        return self.OPs
    
    def __str__(self):
        output = ""
        output = output + "OPs:\n"
        for i in self.OPs:
            output = output + str(i) + "\n"
        output = output + "\nMatrix:\n"
        for i in self.matrix:
            for j in i:
                output = output + str(j) + " "
            output = output + "\n"
        return output
                

In [5]:
def d1_bin(x, bins = 80):

    min_val = np.amin(x)
    max_val = np.amax(x)
    span = max_val - min_val

    p_x = [0.0 for i in range(bins)]

    for i in x:
        bin_num = (int) (bins * (i - min_val) / span)
        if bin_num == bins:
            bin_num -= 1
        p_x[bin_num] += 1.0 / len(x)

    return p_x

In [6]:
def d2_bin(x, y, bins = 80):

    if len(x) != len(y):
        raise Exception("Order parameter lists are of different size.")

    min_x = np.amin(x)
    max_x = np.amax(x)
    span_x = max_x - min_x

    min_y = np.amin(y)
    max_y = np.amax(y)
    span_y = max_y - min_y

    p_xy = [[0.0 for i in range(bins)] for j in range(bins)]

    for i in range(len(x)):
        bin_x = (int) (bins * (x[i] - min_x) / span_x)
        bin_y = (int) (bins * (y[i] - min_y) / span_y)
        if bin_x == bins:
            bin_x -= 1
        if bin_y == bins:
            bin_y -= 1
        p_xy[bin_x][bin_y] += 1.0 / len(x)

    return p_xy

In [7]:
class Memoizer:
    
    def __init__(self):
        self.memo = {}
        self.bins = 20
        
    def iqr(self, OP1, OP2):
        index = str(OP1.name) + " " + str(OP2.name)
        if index in self.memo:
            return self.memo[index]
        else:
            x = OP1.traj
            y = OP2.traj
            p_x = d1_bin(x, self.bins)
            p_y = d1_bin(y, self.bins)
            p_xy = d2_bin(x, y, self.bins)

            info = 0
            entropy = 0

            for i in range(len(p_x)):
                for j in range(len(p_y)):
                    if p_xy[i][j] != 0:
                        entropy -= p_xy[i][j] * np.log(p_xy[i][j])
                        info += p_xy[i][j] * np.log(p_xy[i][j] / (p_x[i] * p_y[j]))

            if ((1 - (info / entropy)) < 0):
                output = 0.0
            else:
                output = (1 - (info / entropy))
            self.memo[index] = output
            return output
        
    def iqr2(self, OP1, OP2):
        index = str(OP1.name) + " " + str(OP2.name)
        x = OP1.traj
        y = OP2.traj
        p_x = d1_bin(x, self.bins)
        p_y = d1_bin(y, self.bins)
        p_xy = d2_bin(x, y, self.bins)
        info = 0
        entropy = 0

        for i in range(len(p_x)):
            for j in range(len(p_y)):
                if p_xy[i][j] != 0:
                    entropy -= p_xy[i][j] * np.log(p_xy[i][j])
                    info += p_xy[i][j] * np.log(p_xy[i][j] / (p_x[i] * p_y[j]))
        return 1 - info/entropy
    
    def __str__(self):
        print(len(self.memo))

In [8]:
def grouping(new_OPs, all_OPs, mut):
    groups = [[] for i in range(len(new_OPs))]
    for OP in all_OPs:
        group = 0
        for i in range(len(new_OPs)):
            tmp = mut.iqr(OP, new_OPs[i])
            if tmp < mut.iqr(OP, new_OPs[group]):
                group = i
        groups[group].append(OP)
    return groups

In [9]:
def group_evaluation(OPs, mut):
    
    center = OPs[0]
    min_distortion = distortion([OPs[0]], OPs, mut)
    
    for i in OPs:
        tmp = distortion([i], OPs, mut)
        if tmp < min_distortion:
            center = i
            min_distortion = tmp

    return center

In [10]:
def cluster(ops, seeds, mut):
    
    old_centers = []
    centers = copy.deepcopy(seeds)
    
    while (set(centers) != set(old_centers)):
        
        old_centers = copy.deepcopy(centers)
        centers = []
        groups = grouping(old_centers, ops, mut)
        for i in range(len(groups)):
            result = group_evaluation(groups[i], mut)
            centers.append(result)

    return centers


In [11]:
def find_ops(old_ops, max_outputs):
    
    mut = Memoizer()
    matrix = DissimilarityMatrix(max_outputs, mut)
    
    for i in old_ops:
        matrix.add_OP(i)
        
    for i in old_ops[::-1]:
        matrix.add_OP(i)
        
    while (len(matrix.OPs) > 20):
        matrix.reduce()
        
    print("Dissimilarity Matrix OPs:")
    for i in matrix.OPs:
        print(i.name)
    
    tmp = copy.deepcopy(matrix)
    distortion_array = []
    num_array = []

    print("\nClustering:")
    while (len(tmp.OPs) > 1):
        print(len(tmp.OPs))
        num_array.append(len(tmp.OPs))
        seed = []
        for i in tmp.OPs:
            seed.append(i)
        tmp_ops = cluster(old_ops, seed, mut)
        print(str(len(tmp.OPs)) + " OPs:")
        for i in tmp_ops:
            print(i.name)
        print("\n")
        distortion_array.append(distortion(tmp_ops, old_ops, mut))
        tmp.reduce()
        
    print("\nDistortions:")
    print(num_array)
    print(distortion_array)
    
    distortion_array = np.array(distortion_array) ** (-0.5)
    
    jumps = []
    
    for i in range(len(distortion_array) - 1):
        jumps.append(distortion_array[i] - distortion_array[i + 1])
    
    min_index = 0
    for i in range(len(jumps)):
        if jumps[i] > jumps[min_index]:
            min_index = i
    
    while (len(matrix.OPs) > num_array[min_index]):
        matrix.reduce()
        
    return cluster(old_ops, matrix.OPs, mut)


In [12]:
f = open("pavanala/COLVAR")

first_line = f.readline()
ops = [OrderParameter(i, []) for i in first_line.split("#! FIELDS time ")[1].split(" ")]


for line in f:
    if "#!" in line:
        continue
    for i in range(len(line.split(' ')) - 2):
        ops[i].traj.append(float(line.split(' ')[i+2]))

f.close()


In [13]:
for i in ops:
    print(i)
final_ops = find_ops(ops, 5)

phi
psi
57_911
119_1517
d2_19
d2_17
d8_18

Dissimilarity Matrix OPs:
d8_18

119_1517
57_911
d2_17
d2_19

Clustering:
5
5 OPs:
d8_18

psi
phi
d2_17
d2_19


4
4 OPs:
d8_18

psi
phi
d2_19


3
3 OPs:
psi
phi
d2_19


2
2 OPs:
psi
phi



Distortions:
[5, 4, 3, 2]
[0.638718882726145, 0.8738971036374119, 1.1617300360416716, 1.4745381175722474]


In [14]:
for i in final_ops:
    print(i.name)

d8_18

psi
phi
d2_17
d2_19
